# Import Module

In [1]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   136k      0 --:--:-- --:--:-- --:--:--  136k
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://storage.googleapis.com/tensorflow-servi

In [2]:
import os
import json
import tempfile
import requests
import subprocess

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist

In [3]:
MODEL_DIR = tempfile.gettempdir()
def gen_export_path(ver):    
    export_path = os.path.join(MODEL_DIR, str(ver))
    return export_path 

In [4]:
os.environ["MODEL_DIR"] = MODEL_DIR
# !ps -ef
# !kill <PID>
# !!killall5 -9

In [5]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


# Build Model & Serve API

In [7]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = np.expand_dims(X_train, -1), np.expand_dims(X_test, -1) # expand (height x weight) to (height x weight x color), color = 1 for grayscale
print(X_train.shape, X_test.shape)

K = len(set(y_train))
print(K)

4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (10000, 28, 28, 1)
10


In [8]:
# Label mapping
labels = '''T-shirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle boot'''.split("\n")

In [9]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   169k      0 --:--:-- --:--:-- --:--:--  169k
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hi

In [ ]:
data = json.dumps({"signature_name": "serving_default", "instances": X_test[0:3].tolist()})
headers = {"content-type": "application/json"}

## Model - Large

In [10]:
i = Input(shape=X_train[0].shape)
x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)

model = Model(i, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 13, 13, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15)

Epoch 1/15
1875/1875 [==============================] - 38s 3ms/step - loss: 0.5201 - accuracy: 0.8074 - val_loss: 0.4095 - val_accuracy: 0.8444
Epoch 2/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3602 - accuracy: 0.8654 - val_loss: 0.3437 - val_accuracy: 0.8707
Epoch 3/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3117 - accuracy: 0.8837 - val_loss: 0.3265 - val_accuracy: 0.8781
Epoch 4/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2790 - accuracy: 0.8953 - val_loss: 0.3136 - val_accuracy: 0.8846
Epoch 5/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2535 - accuracy: 0.9049 - val_loss: 0.2865 - val_accuracy: 0.8980
Epoch 6/15
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2335 - accuracy: 0.9109 - val_loss: 0.2847 - val_accuracy: 0.8956
Epoch 7/15
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2140 - accuracy: 0.9184 - val_loss: 0.2886 - val_accuracy

In [38]:
export_path = gen_export_path(1)
tf.saved_model.save(model, export_path)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /tmp/1/assets


In [39]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0708 16:12:28.099042 140719718954880 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/re

In [ ]:
# Model - Large
r = requests.post('http://localhost:8501/v1/models/fashion_model/versions/1:predict', data=data, headers=headers)     # specifically ver1
p = r.json()['predictions']
p = np.array(p).argmax(axis=1)
p = [labels[p_] for p_ in p]
a = [labels[i] for i in y_test[:3]]
print(p, a)

['Ankle boot', 'Pullover', 'Trouser'] ['Ankle boot', 'Pullover', 'Trouser']


## Model - Tiny

In [14]:
i = Input(shape=X_train[0].shape)
x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
x = Flatten()(x)
x = Dense(K, activation='softmax')(x)

model2 = Model(i, x)
model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 32)        320       
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                54090     
Total params: 54,410
Trainable params: 54,410
Non-trainable params: 0
_________________________________________________________________


In [15]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r2 = model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15)

Epoch 1/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4793 - accuracy: 0.8326 - val_loss: 0.4057 - val_accuracy: 0.8543
Epoch 2/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3414 - accuracy: 0.8793 - val_loss: 0.3462 - val_accuracy: 0.8786
Epoch 3/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3015 - accuracy: 0.8928 - val_loss: 0.3376 - val_accuracy: 0.8793
Epoch 4/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2778 - accuracy: 0.9008 - val_loss: 0.3151 - val_accuracy: 0.8868
Epoch 5/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2609 - accuracy: 0.9064 - val_loss: 0.3086 - val_accuracy: 0.8899
Epoch 6/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2464 - accuracy: 0.9105 - val_loss: 0.3116 - val_accuracy: 0.8889
Epoch 7/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2350 - accuracy: 0.9154 - val_loss: 0.3053 - val_accuracy:

In [42]:
export_path = gen_export_path(2)
tf.saved_model.save(model2, export_path)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /tmp/2/assets


In [43]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_input_2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0708 16:13:05.994067 140186128680832 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/re

In [45]:
# Model - Tiny
# r = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)  # latest version
r = requests.post('http://localhost:8501/v1/models/fashion_model/versions/2:predict', data=data, headers=headers)     # specifically ver2
p = r.json()['predictions']
p = np.array(p).argmax(axis=1)
p = [labels[p_] for p_ in p]
a = [labels[i] for i in y_test[:3]]
print(p, a)

['Ankle boot', 'Pullover', 'Trouser'] ['Ankle boot', 'Pullover', 'Trouser']


In [46]:
# non-existent model will return Response 404
requests.post('http://localhost:8501/v1/models/fashion_model/versions/3:predict', data=data, headers=headers)     # specifically ver2

<Response [404]>

# Conversion to TF Lite

In [59]:
!rm -rf models
!mkdir models

In [60]:
model_tflite = tf.lite.TFLiteConverter.from_keras_model(model).convert()
with open("models/model.tflite", "wb") as f:
    f.write(model_tflite)

INFO:tensorflow:Assets written to: /tmp/tmp23bf8_b2/assets


INFO:tensorflow:Assets written to: /tmp/tmp23bf8_b2/assets


In [61]:
model.save('./models/my_model.h5')

In [62]:
!ls -l models
# tf lite model is much smaller

total 5692
-rw-r--r-- 1 root root 1444956 Jul  8 16:28 model.tflite
-rw-r--r-- 1 root root 4379368 Jul  8 16:28 my_model.h5


In [47]:
import time
while True:
    print('.')
    time.sleep(60)

.
.
.
.
.
.
.


KeyboardInterrupt: ignored